In [1]:
#Read dataframes
import pandas as pd
df_airlines = pd.read_csv('/Users/hannapedersen/Documents/NOVA/T3/Advanced Programming/Assignment/flight_data/airlines.csv')
df_airplanes = pd.read_csv('/Users/hannapedersen/Documents/NOVA/T3/Advanced Programming/Assignment/flight_data/airplanes.csv', on_bad_lines='skip')
df_airports = pd.read_csv('/Users/hannapedersen/Documents/NOVA/T3/Advanced Programming/Assignment/flight_data/airports.csv', on_bad_lines='skip')
df_routes =  pd.read_csv('/Users/hannapedersen/Documents/NOVA/T3/Advanced Programming/Assignment/flight_data/routes.csv', on_bad_lines='skip')


In [10]:
df_airports


,index,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
0,0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7693,7693,14106,Rogachyovo Air Base,Belaya,Russia,\N,ULDA,71.616699,52.478298,272,\N,\N,\N,airport,OurAirports
7694,7694,14107,Ulan-Ude East Airport,Ulan Ude,Russia,\N,XIUW,51.849998,107.737999,1670,\N,\N,\N,airport,OurAirports
7695,7695,14108,Krechevitsy Air Base,Novgorod,Russia,\N,ULLK,58.625000,31.385000,85,\N,\N,\N,airport,OurAirports
7696,7696,14109,Desierto de Atacama Airport,Copiapo,Chile,CPO,SCAT,-27.261200,-70.779198,670,\N,\N,\N,airport,OurAirports


In [12]:
 def departing_flights_airport(airport, internal=False):
        # Join routes with airports to find Source country
        airport_info_1 = df_routes.join(df_airports.set_index('IATA')[['Country']], on='Source airport')
        # Rename the column
        airport_info_1.rename(columns={'Country': 'Source Country'}, inplace=True)
        
        # Join new dataframe with airports to find Destination country
        airport_info_2 = airport_info_1.join(df_airports.set_index('IATA'), on='Destination airport', rsuffix='_dest')
        # Rename the column if needed
        airport_info_2.rename(columns={'Country': 'Destination Country'}, inplace=True)
        # Drop the additional index columns
        airport_info_2 = airport_info_2.reset_index(drop=True)
        

        # Filter flights based on the given source airport
        source_flights = airport_info_2[airport_info_2['Source airport'] == airport]
        print(source_flights)

        if internal:
            # Filter for internal flights (destination in the same country)
            source_flights = source_flights[source_flights['Source Country'] == source_flights['Destination Country']]

        # Check if there are any flights to display
        if not source_flights.empty:
            if internal:
                print(f"Internal flights from {airport} to destinations in the same country:")
            else:
                print(f"All flights from {airport}:")

            print(source_flights[['Source Country', 'Source airport', 'Destination airport', 'Destination Country']])
        else:
                print(f"No internal flights.")
    

In [ ]:
!pip install folium
import folium

In [44]:

def departing_flights_airport(airport, internal=False):
    
    # Join routes with airports to find Source and Destination information
    airport_info_1 = df_routes.join(df_airports.set_index('IATA')[['Country', 'Latitude', 'Longitude']], on='Source airport', rsuffix='_source')
    airport_info_1.rename(columns={'Country': 'Source Country', 'Latitude': 'Source Latitude', 'Longitude': 'Source Longitude'}, inplace=True)
    airport_info_2 = airport_info_1.join(df_airports.set_index('IATA')[['Country', 'Latitude', 'Longitude']], on='Destination airport', rsuffix='_dest')
    airport_info_2.rename(columns={'Country_dest': 'Destination Country', 'Latitude': 'Destination Latitude', 'Longitude': 'Destination Longitude'}, inplace=True)
  
    # Filter flights based on the given source airport
    source_flights = airport_info_2[airport_info_2['Source airport'] == airport]
    source_flights_unique = source_flights.drop_duplicates(subset=['Destination airport'])


    if internal:
        # Filter for internal flights (destination in the same country)
        source_flights_unique = source_flights_unique[source_flights_unique['Source Country'] == source_flights_unique['Destination Country']]
        print(f"Internal flights from {airport} to destinations in the same country:")
    else:
        print(f"All flights from {airport}:")

    # Check if there are any flights to display
    if not source_flights_unique.empty:
        # Create a base map centered around the source airport
        m = folium.Map(location=[source_flights_unique.iloc[0]['Source Latitude'], source_flights_unique.iloc[0]['Source Longitude']], zoom_start=5)
        
        # Plot each route
        for idx, row in source_flights_unique.iterrows():
            source = [row['Source Latitude'], row['Source Longitude']]
            destination = [row['Destination Latitude'], row['Destination Longitude']]
            folium.Marker(source, popup=row['Source airport'], icon=folium.Icon(color="green")).add_to(m)
            folium.Marker(destination, popup=row['Destination airport'], icon=folium.Icon(color="red")).add_to(m)
            folium.PolyLine([source, destination], color="blue", weight=2.5, opacity=1).add_to(m)
        
        # Show the map
        return m
    else:
        print(f"No flights found from {airport}.")
        return None


In [51]:
departing_flights_airport('ATL')

All flights from ATL:


In [49]:
df_routes['Source airport'].value_counts().sort_values(ascending=False)

Source airport
ATL    915
ORD    558
PEK    535
LHR    527
CDG    524
      ... 
NLD      1
CPC      1
CTC      1
FMA      1
WYA      1
Name: count, Length: 3409, dtype: int64

In [18]:
df_routes

,index,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
0,0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2
...,...,...,...,...,...,...,...,...,...,...
67658,67658,ZL,4178,WYA,6334,ADL,3341,NaN,0,SF3
67659,67659,ZM,19016,DME,4029,FRU,2912,NaN,0,734
67660,67660,ZM,19016,FRU,2912,DME,4029,NaN,0,734
67661,67661,ZM,19016,FRU,2912,OSS,2913,NaN,0,734


In [19]:
if 'IATA' not in df_airports.columns:
    df_airports = df_airports.reset_index()

# Then, for joining, use the correct column names and ensure they exist
airport_info_1 = df_routes.join(df_airports.set_index('IATA')[['Country', 'Latitude', 'Longitude']], on='Source airport', rsuffix='_source')

airport_info_2 = airport_info_1.join(df_airports.set_index('IATA')[['Country', 'Latitude', 'Longitude']], on='Destination airport', rsuffix='_dest')

# Check the creation of 'Destination Latitude' and 'Destination Longitude'
print(airport_info_2.columns)

Index(['index', 'Airline', 'Airline ID', 'Source airport', 'Source airport ID',
       'Destination airport', 'Destination airport ID', 'Codeshare', 'Stops',
       'Equipment', 'Country', 'Latitude', 'Longitude', 'Country_dest',
       'Latitude_dest', 'Longitude_dest'],
      dtype='object')


In [ ]:
#df_airport_dest = df_airports.rename(columns ={'Country_dest': 'Destination Country', 'Latitude_dest': 'Destination Latitude', 'Longitude_dest': 'Destination Longitude'})
